## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Edward Day
    - Email: ED558@drexel.edu
- Group member 2
    - Name:Sahar Siddiqi
    - Email: ss5226@drexel.edu
- Group member 3
    - Name: NA
    - Email: NA
- Group member 4
    - Name: NA
    - Email: NA

### Additional submission comments
- Tutoring support received: Sahar Siddiqi ss5226@drexel.edu Jacob Rosen jkr58@drexel.edu Palash Pandey pp535@drexel.edu
- Other (other): NA

# Assignment group 2: Network and exploratory data analysis

## Module D _(40 pts)_ An ingredient-based recommender system
In this module we're going to build a recommender system using some recipes data and the Apriori algorithm. These data can be obtained from Kaggle:

- https://www.kaggle.com/kaggle/recipe-ingredients-dataset

and are packaged with the assignment in the following directory:

- `./data/train.json`

__D1.__ _(2 pts)_ To start, load the recipe data from `json` format and print the first 5 recipes.

In [15]:
import pandas as pd
import numpy as np
import json
import os
from pprint import pprint
import networkx as nx
from networkx.readwrite import json_graph
%matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter
import collections

In [17]:
from pprint import pprint
import json
with open('./data/train.json') as jfile:
    data = json.load(jfile)
pprint(data[:5])

[{'cuisine': 'greek',
  'id': 10259,
  'ingredients': ['feta cheese crumbles',
                  'garlic',
                  'seasoning',
                  'grape tomatoes',
                  'black olives',
                  'garbanzo beans',
                  'pepper',
                  'purple onion',
                  'romaine lettuce']},
 {'cuisine': 'southern_us',
  'id': 25693,
  'ingredients': ['ground pepper',
                  'ground black pepper',
                  'vegetable oil',
                  'plain flour',
                  'thyme',
                  'salt',
                  'green tomatoes',
                  'milk',
                  'yellow corn meal',
                  'eggs',
                  'tomatoes']},
 {'cuisine': 'filipino',
  'id': 20130,
  'ingredients': ['butter',
                  'green chilies',
                  'cooking oil',
                  'chicken livers',
                  'pepper',
                  'salt',
                  'grilled chic

__D2.__ _(5 pts)_ Next, `from collections import Counter` to write a function called `count_items(recipes)` that counts up the number of recipes that include each `ingredient`, storing each in the counter as a single-element tuple (for downstream convienience), i.e., incrementing like `counts[tuple([ingredient])] +=1`. 

When complete, exhibit this functions utility in application to the `recipes` loaded in __D1__ and print the number of 'candidates' in the output.

In [83]:
from collections import Counter
def count_items(recipies):
    frequency = Counter()

 
    for recipie in recipies:
        for ingredient in recipie['ingredients']:
            frequency[tuple([ingredient])] +=1
           
            
            
    return frequency
a = count_items(data)


In [85]:
print(len(a.keys()))


6714


__D3.__ _(5 pts)_ Now, write a function called `store_frequent(candidates, threshold = 25)`, which accepts a `Counter` of `candidates`, i.e., item or itemset counts, and stores only those with count above the determined `threshold` value in a separate counter called `frequent`, which is `return`ed at the end of the function. Apply this function to your output from __D1__ with the default `threshold` value of `25` to exhibit your function's utility, and then print the number of frequent items found.

In [87]:
def store_frequent(candidates, threshold = 25):

    frequent = Counter()
    
    for i in candidates:
        if candidates[i] > threshold:
            frequent[i]=candidates[i]
        
        
    #loop over output from d2 - if the value of the ingredient is higher than the threshold then add it to cand  
  
    return frequent
        
    

In [89]:
freq_cand = store_frequent(a)
print(len(freq_cand.keys()))


1460


__D4.__ (10 pts) Now, write a function called `get_next(recipes, frequent, threshold = 25)` that accepts the `frequent` items output from the `store_frequent()` function. With these inputs, your function should:

1. create a new `Counter` called `next_candidates`
2. compute the `size` of the itemsets for `next_candidates` from a single key in `frequent`
3. `for` any `recipe` with _at least_ as many ingredients as `size`:
    1. loop over all itemsets of size `size` (see combinations note below)
    2. utilize the apriori principle and subsets of itemsets to count up potentially-frequent candidate itemsets in `next_candidates`
4. `return(next_candidates)` 

__Important__: once your code runs, apply this function to the output of __D3__, report the resulting number of `next_candidates` found, and run `store_frequent` on these to report the number of 2-itemsets that were frequent. Repeat this process to build the 3-itemsets and record in the markdown box any observations on run time for these successive applications. In the response box below reply to the following questions:

- Are we generating more candidates or frequent itemsets as we look at larger sizes? 
- Why would this process become more and more computationally expensive as the size get's larger?
    
Note: to complete this part it is _extremely strongly_ encouraged that you import the `combinations()` function from the `itertools` module. With this, you can execute `combinations(items, k)` to find all combinations of size `k` from a list of `items`.

In [91]:

frequent= store_frequent(a)

_Response._

In [93]:
from itertools import combinations
def get_next(recipies, frequent, threshold = 25):
    next_candidates = Counter()
    size = len(list(frequent.keys())[0])
  
    for receipe in recipies:
        if len(receipe['ingredients']) >= size+1: 
            comb = combinations(receipe['ingredients'], size+1)
            for i in comb:
                comba = (i)
                next_candidates[comba]+=1

    
    return next_candidates
    

In [95]:
two_items = get_next(data,freq_cand)
two_items_freq = store_frequent(two_items)
print(len(two_items_freq.keys()))

13952


In [ ]:
#With more entries and more items to choose from, more nodes and connections are expected. Thus, it makes sense
#that the amount of candidates (and by extention frequency) would ultimately increase.

__D5.__ (10 pts) Now that we have the pieces to run Apriori/collect frequent itemsets it's time to package the process together, collecting all frequent itemsets up to a particular `size`. To do this, write a function called `train(recipes, size = 4)`, which:

1. initializes two empty dictionaries, `candidates`, and `frequent`;
2. runs the `count_items` and `store_frequent` function, storing output in the `candidates`, and `frequent` dictionaries using the integer `1` as a key;
3. loops over sizes: 2, 3, .., `size` to compute and store the subsequent sizes candidates and frequent itemsets in the same structure as (2), but now utilizing the `get_next` function, instead of `count_items`; and
4. `return`s the `candidates` and `frequent` itemsets.

In [97]:
def train(recipies, size=4):
    candidates = {}
    frequent = {}

    for i in range (1,size+1):
        if i == 1:
            candidates[i]= count_items(recipies)
            frequent[i] = store_frequent(candidates[i])
        elif i > 1:
            candidates[i] = get_next(recipies, frequent[i-1])
            frequent[i] = store_frequent(candidates[i])
    
    
    return candidates,frequent


    

In [98]:
candidates, frequent  = train(data)
candidates, frequent

({1: Counter({('feta cheese crumbles',): 358,
           ('garlic',): 7380,
           ('seasoning',): 137,
           ('grape tomatoes',): 228,
           ('black olives',): 229,
           ('garbanzo beans',): 148,
           ('pepper',): 4438,
           ('purple onion',): 1896,
           ('romaine lettuce',): 270,
           ('ground pepper',): 385,
           ('ground black pepper',): 4784,
           ('vegetable oil',): 4385,
           ('plain flour',): 154,
           ('thyme',): 361,
           ('salt',): 18048,
           ('green tomatoes',): 108,
           ('milk',): 2263,
           ('yellow corn meal',): 341,
           ('eggs',): 3388,
           ('tomatoes',): 3058,
           ('butter',): 4847,
           ('green chilies',): 768,
           ('cooking oil',): 483,
           ('chicken livers',): 65,
           ('grilled chicken breasts',): 5,
           ('garlic powder',): 1442,
           ('soy sauce',): 3296,
           ('mayonaise',): 781,
           ('yellow onion'

In [100]:
print(len(candidates[1]))
print(len(frequent[1]))
print(len(candidates[2]))
print(len(frequent[2]))
print(len(candidates[3]))
print(len(frequent[3]))
print(len(candidates[4]))
print(len(frequent[4]))

6714
1460
536742
13952
6284926
17091
31701836
5905


__D5.__ _(8 pts)_ Now that we have our `frequent` itemsets up to `size`, we can utilize them to recommend missing ingredients from ingredient 'baskets' of at most `size - 1`. To do this, write a function called `recommend(basket, frequent)` that does the following: 

1. initializes an empty `recommendations` list
2. loops over all frequent `itemset`s of `size 1 greater than the `basket`
    - if there's one item left from the `itemset` when the `basket` removed, append the remaining item to the `recommendations` list in a tuple, with the number of ocurrences of the itemset in the second position
4. `return` `recommendations`, but sorted from high to low by itemset ocurrence.

Once your code is complete, report the top 10 recommended items to buy for recipe flexibility in the following scenarios:

- `basket = tuple(['butter', 'flour'])`
- `basket = tuple(['soy sauce', 'green onions'])`
- `basket = tuple(['avocado', 'garlic', 'salt'])`

and in the response box below discuss the output and the types of recipes you think the recommender is pointing you to. Does this output seem appropriate? 

Note: your function should additionally respond appropriately if the user requests a recommendation for a basket of size at least as big as the `size` specified in the `train()` function, i.e., it should return an error message gracefully, alerting the user to not having trained on itemsets large enough.

_Response._

In [102]:
def recommend(basket,frequent):
    from collections import Counter
    rem = []
    rem_count = Counter()
    recommendations = []
    size = len(basket)
    if size in frequent.keys():
        basket_list = list(basket)
        for x in frequent[size+1]:
            list_freq = list(x)
            remainder = [y for y in list_freq if y not in basket_list]
            if len(remainder) == 1:
                rem.extend(remainder)
        rem_count = Counter(rem)
        recommendations = list(rem_count.items())
        recommendations = sorted(recommendations,key = lambda x: -x[1])
    else:
        print("This program has not been trained on itemsets of the size you entered. Please provide a smaller set.")
    return recommendations
                

In [114]:
basket1 = tuple(['butter','flour'])
rec1 = recommend(basket1,frequent)
print(rec1[:10])

[('salt', 2), ('buttermilk', 1), ('olive oil', 1), ('baking soda', 1), ('onions', 1), ('garlic', 1), ('ground black pepper', 1), ('chicken broth', 1), ('pepper', 1), ('milk', 1)]


In [111]:
basket2 = tuple(['soy sauce', 'green onions'])
rec2 = recommend(basket2,frequent)
print(rec2[:10])

[('sesame oil', 4), ('sugar', 2), ('vegetable oil', 1), ('shiitake', 1), ('chicken breasts', 1), ('rice vinegar', 1), ('red pepper flakes', 1), ('fresh ginger', 1), ('corn starch', 1), ('peanut oil', 1)]


In [107]:
basket3 = tuple(['avocado','garlic','salt'])
rec3 = recommend(basket3, frequent)
print(rec3[:10])

[('olive oil', 1), ('onions', 1), ('lime', 1), ('cilantro', 1), ('pepper', 1), ('sour cream', 1), ('jalapeno chilies', 1), ('black beans', 1)]


In [115]:
#Overall, these items seem very appropriate. For example, when looking at basket3 with the suggestion of avocado,
#garlic and salt, it seems approprite that suggested items include things that go into a taco or Mexican dish.